In [1]:
print("Bismillah Hirrahamaa Nirraheem")

Bismillah Hirrahamaa Nirraheem


In [2]:
import os,sys,warnings
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler,minmax_scale,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression,LogisticRegression,SGDRegressor,SGDClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
# Settings
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Fira Code")))
pio.templates.default = "mod+plotly_dark"
np.set_printoptions(precision=2)
pd.set_option("display.float_format","{:.2f}".format)

# Data Download

In [3]:
# Local data parsing
from zipfile import ZipFile
with ZipFile("/Users/sohai/Downloads/pilot_data.zip","r") as f:
    f.extractall("/Users/sohai/Downloads/pilot_data")
total_raw_df = pd.read_csv("/Users/sohai/Downloads/pilot_data/Airline Dataset.csv")

# Kaggle data parsing
# total_raw_df = pd.read_csv("/kaggle/input/airline-dataset/Airline Dataset.csv")

In [4]:
total_raw_df.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,6/28/2022,CXF,Edithe Leggis,On Time
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,On Time
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,On Time
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,Delayed
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,On Time


In [5]:
total_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98619 entries, 0 to 98618
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Passenger ID          98619 non-null  int64 
 1   First Name            98619 non-null  object
 2   Last Name             98619 non-null  object
 3   Gender                98619 non-null  object
 4   Age                   98619 non-null  int64 
 5   Nationality           98619 non-null  object
 6   Airport Name          98619 non-null  object
 7   Airport Country Code  98619 non-null  object
 8   Country Name          98619 non-null  object
 9   Airport Continent     98619 non-null  object
 10  Continents            98619 non-null  object
 11  Departure Date        98619 non-null  object
 12  Arrival Airport       98619 non-null  object
 13  Pilot Name            98619 non-null  object
 14  Flight Status         98619 non-null  object
dtypes: int64(2), object(13)
memory usage

<font face="Fira Code">
<font size=5>

|SI No|Column Name|Type|Description|
|-----|-----------|----|-----------|
|1|Passenger ID|Categorical(omittable)|Unique identifier for each passenger|Unique identifier for each passenger|
|2|First Name|Categorical(omittable)|First name of the passenger|
|3|Last Name|Categorical(omittable)|Last name of the passenger|
|4|Gender|Categorical|Gender of the passenger|
|5|Age |Continuous| Age of the passenger|
|6|Nationality|Categorical|Nationality of the passenger|
|7|Airport Name|Categorical|Name of the airport where the passenger boarded|
|8|Airport Country Code|Categorical(omittable)|Country code of the airport's location|
|9|Country Name|Categorical| Name of the country the airport is located in|
|10|Airport Continent|Categorical(omittable)| Continent where the airport is situated|
|11|Continents|Categorical|Continents involved in the flight route|
|12|Departure Date|Time Series|Date when the flight departed|
|13|Arrival Airport|Categorical|Destination airport of the flight|
|14|Pilot Name|Categorical(omittable)|Name of the pilot operating the flight|
|15|Flight Status|Categorical|Current status of the flight (e.g., on-time, delayed, canceled)|

In [ ]:
def preprocess(data:pd.DataFrame):
    df = data.copy()
    df.drop(columns=["Passenger ID","First Name","Last Name","Airport Country Code","Airport Continent","Pilot Name"],inplace=True)
    df['Departure Date'] = pd.to_datetime(df['Departure Date'])
    return df

train_df = preprocess(total_raw_df)

# Univariate Analysis